<a href="https://colab.research.google.com/github/harishmuh/Deep-Learning_study/blob/main/Tensorflow_starter_for_beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# **TensorFlow starter for beginner**
---

We will conduct short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

This tutorial is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. To run all the code in the notebook, select **Runtime** > **Run all**. To run the code cells one at a time, hover over each cell and select the **Run cell** icon.

![Run cell icon](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/images/beginner/run_cell_icon.png?raw=1)

## **Initial Set up**

We will setup TensorFlow by importing the library:

In [ ]:
# Importing TensorFlow
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.19.0


## **Loading dataset**

Load and prepare the MNIST dataset. The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This also converts the sample data from integers to floating-point numbers:

In [ ]:
# Loading dataset
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## **Building neural network model**

Build a `tf.keras.Sequential` model:

In [ ]:
# Building neural network
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


[`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers.

This model architecture uses
* [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten),
* [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense),  
* [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [ ]:
# Prediction
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.17411666,  0.26184556,  0.03014442, -0.162366  , -0.5110792 ,
         1.0033199 , -0.41995224, -0.5791118 ,  0.4574099 ,  0.2358529 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to *probabilities* for each class:

In [ ]:
tf.nn.softmax(predictions).numpy()

array([[0.07363148, 0.11386742, 0.09031771, 0.07450179, 0.05256813,
        0.23901089, 0.05758355, 0.04911072, 0.13846244, 0.11094584]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparseCategoricalCrossentropy`:

In [ ]:
# Defining loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

np.float32(1.4312462)

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [ ]:
# Compiling model
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## **Training and evaluation**

Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [ ]:
# Training
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8575 - loss: 0.4773
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9561 - loss: 0.1508
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9673 - loss: 0.1098
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9735 - loss: 0.0862
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9780 - loss: 0.0696


The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [ ]:
# Evaluation
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - 2ms/step - accuracy: 0.9754 - loss: 0.0792


[0.07916442304849625, 0.9753999710083008]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.5345170e-10, 6.8753767e-11, 2.2076985e-07, 9.8696746e-06,
        9.9928843e-13, 2.1394952e-09, 4.0985493e-15, 9.9998486e-01,
        1.4212593e-08, 5.0331250e-06],
       [4.9960511e-09, 1.4507399e-05, 9.9995828e-01, 2.2218781e-05,
        1.3926512e-19, 3.0031936e-06, 1.6695866e-10, 1.3130321e-15,
        2.0502787e-06, 3.7819029e-11],
       [1.6875808e-07, 9.9857557e-01, 4.3666154e-05, 3.1049163e-05,
        1.9958324e-04, 8.6280488e-07, 5.6533995e-06, 6.4615806e-04,
        4.9307995e-04, 4.3078207e-06],
       [9.9976295e-01, 9.0278149e-07, 3.7840582e-05, 2.0624169e-07,
        4.5772495e-08, 4.5972618e-07, 3.9860824e-06, 1.7947880e-04,
        6.9124811e-10, 1.4224620e-05],
       [3.3304653e-07, 2.2407177e-11, 1.2412119e-06, 4.1108970e-09,
        9.9897158e-01, 7.1378999e-08, 3.1754907e-06, 5.5679990e-05,
        2.1090108e-07, 9.6769817e-04]], dtype=float32)>

## **Conclusion**

We have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.


